# WaterTAP — Steam-Driven Crystallizer (Industry Demo)
section A: Digital twin of a forced-circulation crystallizer heated by steam: properties → equipment → connections → costing → operating → simulate baseline → optimize → KPIs.

## Step 1 — Import project flowsheet

In [ ]:
import logging, warnings, io
import idaes.logger as idaeslog
for _n in ("idaes","idaes.core.util.scaling","pyomo","watertap"):
    try: idaeslog.getLogger(_n).setLevel(idaeslog.CRITICAL)
    except Exception: logging.getLogger(_n).setLevel(logging.CRITICAL)
logging.getLogger().setLevel(logging.CRITICAL)
warnings.filterwarnings("ignore")

# core imports
import crystallizer_live_steam_with_condenser_chiller as steam
from pyomo.common.log import LoggingIntercept
from IPython.display import Image, display
import os
for p in ("fc.png", "/mnt/data/fc.png"):
    if os.path.exists(p):
        display(Image(p, width=560))
        break
else:
    print("Place 'fc.png' next to this notebook (or /mnt/data).")
print("✅ Module imported")

## Step 2 — Build the flowsheet (how it’s constructed)
• **Add properties (thermophysical models):** brine (NaCl–H₂O) for the feed, mixed recycle, and purge streams; water/steam for vapor/condensate; a crystallizer salt package for **enthalpy** and **phase equilibrium/solubility** to represent supersaturation and crystal formation.  
• **Add unit models (equipment):** Feed & **Mixer** (blend with recycle), **Pump** (forced circulation), **Steam Heater** (hot-side steam / cold-side brine), **Crystallizer** (crystal growth + vapor disengagement), **Condenser + Chiller** (recover water and close the cold loop), **Separator** (split slurry into recycle + purge), **Product** (distillate).  
• **Add connections (arcs):** pipe unit ports in process order; route vapor to condenser; return condenser cold outlet via chiller; create recycle and purge branches like a plant PFD.  
• **Add practical constraints:**  
  – **Max temperature rise across the liquor in the heater** to moderate supersaturation rate, avoid wall-scale/flash boiling, and protect the exchanger—typically **~3–6 K** (site-tuned).  
  – **ΔTmin check** (steam–liquor approach) for heat-exchanger design sanity.
• **purge stream:** a small, controlled branch removes concentrated liquor/solids to keep the loop inside scaling limits and keep product quality steady—just like FC circuits in the field.  
• **Add costing:** cost blocks on key units so physics maps directly to **LCOW** and energy KPIs.

In [ ]:
# Build (logs suppressed)
with LoggingIntercept(io.StringIO(), level=logging.CRITICAL):
    m = steam.build()
print("✅ Flowsheet built")
# --- Example snippet (comments only)
# # m.fs.brine_props = ...
# # m.fs.mixer = ...; m.fs.pump = ...; m.fs.heater = ...; m.fs.cryst = ...
# # m.fs.condenser = ...; m.fs.chiller = ...; m.fs.separator = ...
# # m.fs.s01 = Arc(...);  # etc.

## Step 3 — Set operating conditions (datasheet values)
**What gets specified:** feed composition/temperature/pressure, pump ΔP for circulation, crystallizer operating temperature, steam-side conditions.  

In [ ]:
steam.set_operating_conditions(m)
print("✅ Operating conditions applied")
# --- Example snippet (comments only)
# # m.fs.feed.flow_mass_phase_comp[0,"Liq","NaCl"].fix(...)
# # m.fs.feed.temperature[0].fix(...); m.fs.pump.deltaP.fix(...)
# # m.fs.cryst.temperature_operating.set_value(...)
# # steam-side specs on heater; condenser/chiller temperatures/UA/area

## Step 4 — Initialize (steady-state startup sequence) - (fully specified, DOF = 0)
Bring units online in process order and align states across connections so the flowsheet starts from a stable, plant-sensible point before solving.

In [ ]:
with LoggingIntercept(io.StringIO(), level=logging.CRITICAL):
    steam.initialize_system(m)
print("✅ Initialization complete")

## Step 5 — Set optimization (steady state, with bounds)
Promote selected operating variables to **decisions** and apply **bounds** (e.g., heater outlet-temperature band, circulation targets).  
This remains a steady-state problem but opens degrees of freedom to seek **best achievable cost/energy** under practical limits.

In [ ]:
if hasattr(steam, "optimize_set_up"):
    steam.optimize_set_up(m)   # e.g., objective = LCOW + bounds on key decisions
    print("✅ Optimization setup applied (steady state, bounded DOFs)")
else:
    print("ℹ️ No optimization setup exposed; skipping optimization step.")

## Step 6 — Optimize (steady state)
Solve the bounded, steady-state optimization to find an economical/energy-efficient operating point within plant-sensible limits.

In [ ]:
if hasattr(steam, "optimize_set_up"):
    with LoggingIntercept(io.StringIO(), level=logging.CRITICAL):
        steam.solve(m)
    print("✅ Optimization solved")
else:
    print("ℹ️ Optimization not configured; showing baseline KPIs.")

## Step 8 — KPIs (unit-labeled, plant-relevant)
• **Feed (kg/s)** — incoming mass rate to the loop  
• **Distillate (kg/s)** — clean water produced  
• **Solids NaCl (kg/s)** — crystal production rate  
• **Steam (kg/s)** — motive steam to the heater  
• **Heat duty (MW)** — steam-side duty at the heater  
• **Heater area (m²)** — exchanger size implication  
• **LCOW ($/m³)** — levelized cost signal for decision-making

In [ ]:
from pyomo.environ import value
import pandas as pd

def kpis_mass_energy(m):
    # Feed (kg/s): Liq H2O + Liq NaCl
    feed_kg_s = (
        value(m.fs.feed.flow_mass_phase_comp[0, "Liq", "H2O"])
        + value(m.fs.feed.flow_mass_phase_comp[0, "Liq", "NaCl"])
    )

    # Distillate (kg/s)
    if hasattr(m.fs, "distillate"):  # Steam-Driven & MVC
        d = m.fs.distillate.properties[0]
        try:
            dist_kg_s = value(d.flow_mass_phase_comp["Liq", "H2O"])
        except Exception:
            dist_kg_s = value(d.flow_mass_phase_comp["Vap", "H2O"])
    else:  # TVC: sum heater+condenser product
        def _water_mass(sb):
            try:
                return value(sb.flow_mass_phase_comp["Liq", "H2O"])
            except Exception:
                return value(sb.flow_mass_phase_comp["Vap", "H2O"])
        dh = m.fs.distillate_heater.properties[0]
        dc = m.fs.distillate_condenser.properties[0]
        dist_kg_s = _water_mass(dh) + _water_mass(dc)

    # Solids NaCl (kg/s)
    solids_kg_s = value(m.fs.crystallizer.solids.flow_mass_phase_comp[0, "Sol", "NaCl"])

    # Steam (kg/s) – live steam to heater if present (MVC may not use it)
    steam_kg_s = (
        value(m.fs.heater.hot_side_inlet.flow_mass_phase_comp[0, "Vap", "H2O"])
        if hasattr(m.fs.heater, "hot_side_inlet") else float("nan")
    )

    # Heat duty (MW) & heater area (m²)
    heat_MW   = value(m.fs.heater.hot.heat[0]) / 1e6
    area_m2   = value(m.fs.heater.area) if hasattr(m.fs.heater, "area") else float("nan")

    # LCOW ($/m³)
    lcow = value(m.fs.costing.LCOW)

    return pd.DataFrame([{
        "Feed (kg/s)":        feed_kg_s,
        "Distillate (kg/s)":  dist_kg_s,
        "Solids NaCl (kg/s)": solids_kg_s,
        "Steam (kg/s)":       steam_kg_s,
        "Heat duty (MW)":     heat_MW,
        "Heater area (m²)":   area_m2,
        "LCOW ($/m³)":        lcow,
    }])


kpi_table = kpis_mass_energy(m)
kpi_table.style.format({
    "Feed (kg/s)":"{:.3f}",
    "Distillate (kg/s)":"{:.3f}",
    "Solids NaCl (kg/s)":"{:.3f}",
    "Steam (kg/s)":"{:.3f}",
    "Heat duty (MW)":"{:.3f}",
    "Heater area (m²)":"{:.2f}",
    "LCOW ($/m³)":"{:.2f}",
})

## Section B — Fast-Build MVC (Mechanical Vapor Recompression)
**What’s added vs steam-driven:**  
• **Compressor** mechanically recompresses vapor instead of using more live steam.  
• Condenser/heat-exchange loop returns latent heat to the brine; electricity becomes the primary energy driver.  
**Connections:** crystallizer vapor → **compressor**  → back into the heater hot side; product water leaves the hot side of heater.  
**Plant intuition:** lower steam use, higher kWh; good where electricity is cheap or steam is constrained.

In [ ]:

from IPython.display import Image, display
import os, io
from pyomo.common.log import LoggingIntercept
if os.path.exists("mvc.png"):
    display(Image("mvc.png", width=560))

import Crystallizer_MVR as mvc


with LoggingIntercept(io.StringIO(), level=logging.CRITICAL):
    m_mvc = mvc.build()
    mvc.set_operating_conditions(m_mvc)
    mvc.initialize_system(m_mvc)
    if hasattr(mvc, "optimize_set_up"):
        mvc.optimize_set_up(m_mvc)  
    mvc.solve(m_mvc)
print("✅ MVC flowsheet solved")

## Section C — Fast-Build TVC (Steam Ejector / Thermal Vapor Compression)
**What’s added vs steam-driven:**  
• **Steam ejector (TVC)** uses **motive steam** to draw and recompress vapor, cutting net external steam to the heater.  
**Connections:** crystallizer vapor → **steam ejector** (plus motive steam) → condenser loop → back to heater; distillate still from condenser.  
**Plant intuition:** mid-ground between steam-only and MVC—still uses steam, but leverages ejector mixing to reduce overall duty.

In [ ]:

if os.path.exists("tvc.png"):
    display(Image("tvc.png", width=560))

import crystallizer_TVC as tvc


with LoggingIntercept(io.StringIO(), level=logging.CRITICAL):
    m_tvc = tvc.build()
    tvc.set_operating_conditions(m_tvc)
    tvc.initialize_system(m_tvc)
    if hasattr(tvc, "optimize_set_up"):
        tvc.optimize_set_up(m_tvc)  
    tvc.solve(m_tvc)
print("✅ TVC flowsheet solved")

## Section D — KPI comparison (Steam-Driven vs MVC vs TVC)
**LCOW** is the headline metric for decision-making.

In [ ]:
from pyomo.environ import value

def kpis_generic(model):
    # reuse mass/energy from earlier; keep LCOW included
    row = kpis_mass_energy(model).iloc[0].to_dict()
    return row

rows = [
    {"Flowsheet":"Steam-Driven", **kpis_generic(m)},
    {"Flowsheet":"MVC (MVR)",   **kpis_generic(m_mvc)},
    {"Flowsheet":"TVC (Ejector)", **kpis_generic(m_tvc)},
]
df_kpis = pd.DataFrame(rows).set_index("Flowsheet")
df_kpis

## Section E — LCOW comparison (bar)
LCOW summarizes the combined effect of energy, throughput, and operating assumptions—ideal for quick technology screening.

In [ ]:
_ = (df_kpis[["LCOW ($/m³)"]]
     .sort_values("LCOW ($/m³)")
     .style.format({"LCOW ($/m³)": "{:.2f}"})
     .bar(subset=["LCOW ($/m³)"], color="#1f77b4"))
_

## Section F — Energy-price sweeps for technology screening
WaterTAP sweeps **steam price ($/ton)** and **electricity price (¢/kWh)** across ranges to compute **LCOW** for each flowsheet (Steam-Driven, MVC, TVC), revealing:
- **LCOW vs steam price**: how each option’s cost moves with steam.  
- **Least-cost technology map (steam vs electricity)**: which option is lowest-cost at a given price pair and the **switch lines** between them.  
- **LCOW surface heatmap**: sensitivity of minimum LCOW to concurrent steam and electricity price changes.  
Use these to quickly **screen technologies**, plan **energy contracts**, and understand **price-exposure risk**.

In [ ]:
from IPython.display import Image, display
from pathlib import Path

for fn in ("steam_cost_vs_lcow.png","steam-electricity-heat-map.png","least-cost-map.png",
           "/mnt/data/steam_cost_vs_lcow.png","/mnt/data/steam-electricity-heat-map.png","/mnt/data/least-cost-map.png"):
    if Path(fn).exists():
        display(Image(fn, width=900))

# --- How to reproduce (commented)
# from sweep_analysis_and_plots import main              
# main()
#
# # Or run individual sweeps (time-consuming):
# from sweep_electricity import run_electricity_price_sweep      # MVC electricity sweep
# from sweep_steam import run_steam_cost_sweep                   # TVC steam sweep
# from sweep_steam_no_heat_recovery import run_steam_cost_sweep  # FC steam sweep
# run_electricity_price_sweep(nx=200, output_filename="electricity_price_sweep_elec.csv")
# run_steam_cost_sweep(nx=200, output_filename="steam_price_sweep_hr.csv")
# run_steam_cost_sweep(nx=200, output_filename="steam_price_sweep_nhr.csv")
# --------------------------------------------------------------------------------